In [ ]:
# ann_model.py

import os, random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Set reproducible seeds
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(42)

# Load dataset
df = pd.read_csv('original_dataset.csv')
X = df[['FTC(No.)', 'CS(MPa)', 'W/C', 'MinT(C)', 'MaxT(C)', 'AE(0/1)']].values
y = df[['DCS(MPa)']].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# Define ANN model
model = Sequential([
    Input(shape=(6,)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile
model.compile(optimizer=Adam(learning_rate=0.002), loss='mse', metrics=['mae'])

# Train
early_stop = EarlyStopping(patience=30, restore_best_weights=True, monitor='val_loss')
history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_test_scaled, y_test_scaled),
    epochs=500,
    batch_size=8,
    verbose=0,
    callbacks=[early_stop]
)

# Plot training curve
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Training History")
plt.legend()
plt.grid(True)
plt.show()

# Evaluation
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test_scaled, y_pred)
rmse = mean_squared_error(y_test_scaled, y_pred, squared=False)
r2 = r2_score(y_test_scaled, y_pred)

print(f"\nANN Performance:")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

# model.save("ann_model.keras")